## 01. 머신러닝 실습

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [0]:
## 키, 몸무게, 레이블이 적힌 CSV 파일 읽어오기
csv = pd.read_csv("bmi.csv")

print(csv.shape)
print(csv.info())
print(csv.describe())
csv.head()

(20000, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
height    20000 non-null int64
weight    20000 non-null int64
label     20000 non-null object
dtypes: int64(2), object(1)
memory usage: 468.8+ KB
None
             height        weight
count  20000.000000  20000.000000
mean     159.927200     57.535000
std       23.342096     13.285259
min      120.000000     35.000000
25%      140.000000     46.000000
50%      160.000000     58.000000
75%      180.000000     69.000000
max      200.000000     80.000000


,height,weight,label
0,142,62,fat
1,142,73,fat
2,177,61,normal
3,187,48,thin
4,153,60,fat


In [0]:
## 데이터 정규화
csv['height'] = csv['height'] / 200
csv['weight'] = csv['weight'] / 100
csv.head(2)

,height,weight,label
0,0.71,0.62,fat
1,0.71,0.73,fat


In [0]:
csv['label'].value_counts()

fat       7322
thin      6393
normal    6285
Name: label, dtype: int64

In [0]:
### 레이블을 배열로 변환 
### thin - (1,0,0), normal - (0,1,0), fat - (0,0,1)
bclass = {"thin":[1,0,0], "normal":[0,1,0], "fat":[0,0,1]}
csv['label_pat'] = csv['label'].apply(lambda x:np.array(bclass[x]))
csv.head()

,height,weight,label,label_pat
0,0.710,0.62,fat,"[0, 0, 1]"
1,0.710,0.73,fat,"[0, 0, 1]"
2,0.885,0.61,normal,"[0, 1, 0]"
3,0.935,0.48,thin,"[1, 0, 0]"
4,0.765,0.60,fat,"[0, 0, 1]"


In [0]:
test_csv = csv[15000:20000]
test_pat = test_csv[ ['weight', 'height']  ]
test_ans = list(test_csv['label_pat'])
print(test_pat.head())
print(test_ans[1:5])

       weight  height
15000    0.55   0.690
15001    0.36   0.760
15002    0.72   0.915
15003    0.51   0.990
15004    0.47   0.745
[array([1, 0, 0]), array([0, 1, 0]), array([1, 0, 0]), array([0, 1, 0])]


In [0]:
## 01. 플레이스 홀더 
x = tf.placeholder(tf.float32, [None, 2])  ## 키와 몸무게 데이터 넣기
y_ = tf.placeholder(tf.float32, [None, 3]) ## 정답 레이블 넣기

## 02. 변수 선언, W, b
W = tf.Variable(tf.zeros([2,3]))  # 가중치
b = tf.Variable(tf.zeros([3]))    # 편향(바이어스)

## 03. 신경망 정의(회귀식 정의) - 다항분류이기때문에 
model_pred = tf.nn.softmax( tf.matmul(x, W) + b )

In [0]:
## 04. cost, optimizer
cross_entropy = -tf.reduce_sum(y_ * tf.log(model_pred))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

## 05. 훈련 

## 06. 정답률 구하기
predict = tf.equal(tf.argmax(model_pred, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

In [0]:
## 세션을 설정하고, 학습을 수행하겠다.
sess = tf.Session()
sess.run(tf.global_variables_initializer() )  # 변수 초기화
# 학습 시키기
for step in range(3500):    # 0,    1,2,3,~ 3499
  i = (step * 100) % 14000  # 0,    100, 200, 300,....
  rows = csv[1+i:1+i+100]   # 1~100, 101~200            #  배치사이즈 
  x_pat = rows[ ['weight', 'height']  ]
  y_ans = list(rows['label_pat'])
  
  fd = {x:x_pat, y_:y_ans}
  sess.run(train, feed_dict=fd)
  if step%500==0:
    # loss(손실), 정확도
    loss = sess.run(cross_entropy, feed_dict=fd)
    acc = sess.run(accuracy, feed_dict={x:test_pat, y_:test_ans})
    print("step =", step, "loss =", loss, "acc =", acc)

step = 0 loss = 108.66269 acc = 0.3242
step = 500 loss = 57.58866 acc = 0.8904
step = 1000 loss = 45.02092 acc = 0.898
step = 1500 loss = 41.654335 acc = 0.9566
step = 2000 loss = 34.664024 acc = 0.943
step = 2500 loss = 34.28702 acc = 0.9674
step = 3000 loss = 26.880762 acc = 0.9726
